#### Combine the data from the folders into a single CSV file.

In [1]:
import pandas as pd
import os

data_dir = 'Data'
speeds = ['Low speed', 'Medium speed', 'High speed'] # Speed categories.

# List to store all the Dataframes
dfs = []

for speed in speeds:
    speed_dir = os.path.join(data_dir, speed)
    for file in os.listdir(speed_dir):
        if file.endswith('.csv'):
            file_path = os.path.join(speed_dir, file)
            df = pd.read_csv(file_path, skiprows=17)
            df = df[['timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz']]
            # Extract weight from the file name.
            node_id = file.split('-')[0]
            weight = file.split('-')[1].replace('kg.csv', '')
            
            df_new = pd.DataFrame({'speed': [speed]*len(df), 'weight': [weight]*len(df), 'node-id': [node_id]*len(df)})
            # Concatenate the new DataFrame with the original DataFrame
            df = pd.concat([df_new, df], axis=1)
            # Remove the extra rows to have exactly 5 minutes (200 samples/s * 5 min = 60000 samples)
            if len(df) > 60000:
                drop_rows = len(df) - 60000
                df = df.iloc[drop_rows:]
            # Reset the index
            df = df.reset_index(drop=True)

            # Append the DataFrame to the list
            dfs.append(df)

# Concatenate all the DataFrames
df_combined = pd.concat(dfs, ignore_index=True)

In [2]:
# Define the order of the categories
df_combined['speed'] = pd.Categorical(df_combined['speed'], categories=['Low speed', 'Medium speed', 'High speed'], ordered=True)

df_combined['weight'] = df_combined['weight'].astype(float)

# Sort the DataFrame based on 'speed' and 'weight'
df_combined = df_combined.sort_values(by=['speed', 'weight'])

# Define the mapping
speed_mapping = {'Low speed': 0, 'Medium speed': 1, 'High speed': 2}

# Replace the 'speed' column with the mapped values
df_combined['speed'] = df_combined['speed'].map(speed_mapping)

df_combined = df_combined.reset_index(drop=True)

# Save the combined DataFrame to a CSV file
df_combined.to_csv('Data/combined_data.csv', index=False)